# AutoUi

With ipyautoui we can create ipywidgets from either a **json-schema** or a **pydantic model**. This makes it quick and easy to whip up a user interface when required.

In [1]:
from ipyautoui import AutoUi
import json
from pydantic import BaseModel, Field
from ipyautoui.constants import DIR_MODULE
from ipyautoui._utils import display_pydantic_json
import ipyautoui
import ipywidgets as w

### Creating Simple Widget

In [2]:
from ipyautoui.demo_schemas import (
    CoreIpywidgets,
)  # the schema shown in the file above

So let's create a simple pydantic class. Here we have one text field.

In [3]:
# create a pydantic model (or a json-schema) defining the fields of interest
from pydantic import BaseModel, Field, model_validator
import pathlib
import typing as ty
from enum import Enum

class My(Enum):
    a= "a"
    b= "b"


class Sub(BaseModel):
    a: str = "a"
    b: int = 1


class Example(BaseModel):
    text: str = Field(default="Test", description="This description is very important")
    inty: int = 1
    sub: Sub
    nu: ty.Optional[str] = None
    my: My


data = {"text": "this is a value"}
ui = AutoUi(
    schema=Example,
    path=pathlib.Path("test.ui.json"),
    show_savebuttonbar=True,
)
display(ui)

AutoUi(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style='success', disabled=True, icon…

In [4]:
# update the `value` of the form as you would any other ipywidget
ui.value = {'text': 'Test upates', 'inty': 1, 'sub': {'a': 'asdf', 'b': 3}, 'nu': "not None", 'my': 'b'}

## Updating the model

Minor changes to the model can be made. The keys and resulting widgets must remain the same, but the attributes that define them can change (i.e. descriptions, the options of Dropdown, limits of an integer etc.).
You can also add validation logic in this way. This is to support the edge case where the parameters of a form must remain the same but the allowed values can change.


In [6]:
class New(Enum):
    "asdfal;ksdfj"
    c = 1
    d = 2
    
class Example2(BaseModel):
    text: str = Field(default="Test", description="This description is very important")
    inty: int = Field(default=4, description="This integer is very important")
    sub: Sub
    nu: ty.Optional[str] = None
    my: New

    @model_validator(mode='after')
    def v(self):
        self.text = "custom validation stuff..."
        return self
    

ui.model = Example2

In [7]:
ui.value

{'text': 'custom validation stuff...',
 'inty': 1,
 'sub': {'a': 'asdf', 'b': 3},
 'nu': 'not None',
 'my': 1}

In [10]:
ui.di_widgets_value

{'text': 'Test', 'inty': 4, 'sub': {'a': 'asdf', 'b': 3}, 'nu': None, 'my': 1}

Wholesale updates of the model are not allowed.

In [6]:
# this will fail as the model is completely different. 
# in this case you just want to new AutoUi object. 
try:
    ui.model = Sub
except Exception as e:
    print(e)

In [5]:
# this will fail as the widgets that define the model need to change
# as `inty` has changed from an `int` to a `str`
class Example1(BaseModel):
    text: str = Field(default="Test", description="This description is very important")
    inty: str = "1"
    sub: Sub
    nu: ty.Optional[str] = None
    my: My

try:
    ui.model = Example1
except Exception as e:
    print(e)

widgets must match on schema change. changes intended for modifications of existing widgets only.


In [8]:
ui.di_widgets['my'].options = [1,2,3]

In [5]:
from ipyautoui.autoobject import AutoObjectForm

AutoObjectForm.from_jsonschema(schema=Example.model_json_schema())

AutoObjectForm(children=(HBox(children=(ToggleButton(value=False, icon='plus', layout=Layout(display='', width…

In [ ]:
from ipyautoui.basemodel import file

file(Example(), pathlib.Path("test.json"))

### Writing to JSON

Let's define the save location.

In [ ]:
import pathlib

save_path = pathlib.Path(".") / "test.simpleaui.json"
print(f"Save Location is: {save_path}")

In [ ]:
ui.file(path=save_path)
AutoUiRenderer = AutoUi.create_autoui_renderer(schema=Example)

ui_simple = AutoUiRenderer(path=save_path)


def test_action():
    print("done")


ui_simple.savebuttonbar.fns_onsave_add_action(test_action)
ui_simple.show_savebuttonbar = True
display(ui_simple)

In [ ]:
import typing as ty
import ipywidgets as w


class DataFrameCols(BaseModel):
    string: str = Field("string", json_schema_extra=dict(column_width=100))
    integer: int = Field(1, json_schema_extra=dict(column_width=80))
    floater: float = Field(3.1415, json_schema_extra=dict(column_width=70, global_decimal_places=3))
    something_else: float = Field(324, json_schema_extra=dict(column_width=100))


class TestDataFrame(BaseModel):
    """a description of TestDataFrame"""

    dataframe: ty.List[DataFrameCols] = Field(json_schema_extra=dict(format="dataframe"))

In [ ]:
auto_grid = AutoUi(schema=TestDataFrame, show_nested=True)
display(auto_grid)

In [ ]:
auto_grid.value = {
    "dataframe": [
        {
            "string": "important string",
            "integer": 1,
            "floater": 3.14,
            "something_else": 324,
        },
        {"string": "update", "integer": 4, "floater": 3.12344, "something_else": 123},
        {"string": "evening", "integer": 5, "floater": 3.14, "something_else": 235},
        {"string": "morning", "integer": 5, "floater": 3.14, "something_else": 12},
        {"string": "number", "integer": 3, "floater": 3.14, "something_else": 123},
    ]
}

In [ ]:
from pydantic import BaseModel, Field
from ipyautoui.custom.fileupload import AutoUploadPaths
from ipyautoui.autoobject import AutoObject
import pathlib


class Test(BaseModel):
    paths: list[pathlib.Path] = Field(
        json_schema_extra=dict(autoui="ipyautoui.custom.fileupload.AutoUploadPaths")
    )
    a: str

    class Config:
        schema_extra = {
            "json_schema_extra": ["ipyautoui.custom.fileupload.AutoUploadPaths"]
        }


aui = AutoUi(Test)  # , nested_widgets=[AutoUploadPaths]
aui

In [ ]:
from ipyautoui import AutoUi
from ipyautoui.demo_schemas import NestedEditableGrid

ui = AutoUi(NestedEditableGrid)
ui